In [1]:
import os

In [20]:
from dotenv import load_dotenv
load_dotenv()

True

In [21]:

GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")

In [22]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT

In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

### Concept of tools

In [3]:
api_wrapper = WikipediaAPIWrapper()

In [4]:
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [5]:
tool.name

'wikipedia'

In [6]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [7]:
tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [8]:
tool.return_direct

False

In [9]:
print(tool.run({"query": "langchain"}))

Page: LangChain
Summary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.

Page: DataStax
Summary: DataStax, Inc. is a real-time data for AI company based in Santa Clara, California. Its product Astra DB is a cloud database-as-a-service based on Apache Cassandra. DataStax also offers DataStax Enterprise (DSE), an on-premises database built on Apache Cassandra, and Astra Streaming, a messaging and event streaming cloud service based on Apache Pulsar. As of June 2022, the company has roughly 800 customers distributed in over 50 countries.



Page: Retrieval-augmented generation
Summary: Retrieval augmented generation (RAG) is a type of generative artificial intelligence that has information retrieval capabilities. It modifies inter

In [10]:
from langchain_community.tools import YouTubeSearchTool

In [11]:
tool=YouTubeSearchTool()

In [12]:
tool.name

'youtube_search'

In [13]:
tool.description

'search for youtube videos associated with a person. the input to this tool should be a comma separated list, the first part contains a person name and the second a number that is the maximum number of video results to return aka num_results. the second part is optional'

In [15]:
tool.run("sunny savita")

"['https://www.youtube.com/watch?v=Ik8gNjJ-13I&pp=ygUMc3Vubnkgc2F2aXRh', 'https://www.youtube.com/watch?v=6GnsndNL5u4&pp=ygUMc3Vubnkgc2F2aXRh']"

In [16]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [23]:
tool = TavilySearchResults()

In [24]:
tool.invoke({"query": "What happened in the latest burning man floods"})

[{'url': 'https://www.npr.org/2023/09/03/1197497458/the-latest-on-the-burning-man-flooding',
  'content': "There are also reports that at least one person has died at the counterculture festival about a hundred miles north of Reno, Nev. Earlier this afternoon, I caught up with NPR's Claudia Peschiutta, who's at her first burn, and she told me it's muddy where she is, but that she and her camp family have been making the best of things.\n National\nThe latest on the Burning Man flooding\nClaudia Peschiutta\nAuthorities are investigating a death at the Burning Man festival in the Nevada desert after tens of thousands of people are stuck in camps because of rain.\n SCOTT DETROW, HOST:\nKnee-deep mud, warnings to conserve food and water, orders to shelter in place - this is all at Burning Man 2023 after torrential rains turned the Black Rock Desert into miles and miles of mud. I mean, mostly what I've seen from my personal experience is just any sort of need that you have, somebody, whethe

In [25]:
from langchain.agents import tool
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

In [26]:
get_word_length.invoke("abc")

3

In [27]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [28]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [32]:
multiply.invoke({"a":10,"b":20})

200

### Concept of Agents

### Load the model

In [36]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.0-pro")

In [43]:
'''from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")'''
from langchain_groq import ChatGroq
import os
llm=ChatGroq(model_name="Gemma2-9b-It")

### This Agent class from the first version of LangChain

In [44]:
from langchain.agents import AgentType
from langchain.agents import load_tools 
from langchain.agents import initialize_agent

In [45]:
tool=load_tools(["wikipedia"],llm=llm)

In [46]:
agent=initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

In [47]:
agent.run("What is current GDP of India?")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Thought: I need to look up India's GDP on Wikipedia. 
Action: wikipedia
Action Input: India economy
Observation: Page: Economy of India
Summary: The economy of India is a developing mixed economy with a notable public sector in strategic sectors. It is the world's fifth-largest economy by nominal GDP and the third-largest by purchasing power parity (PPP); on a per capita income basis, India ranked 136th by GDP (nominal) and 125th by GDP (PPP). From independence in 1947 until 1991, successive governments followed the Soviet model and promoted protectionist economic policies, with extensive Sovietization, state intervention, demand-side economics, natural resources, bureaucrat driven enterprises and economic regulation. This is characterised as dirigism, in the form of the Licence Raj. The end of the Cold War and an acute balance of payments crisis in 1991 led to the adoption of a broad economic liberalisation in India and indicative planning. Since the start of the 21st century, annual 

"The current GDP of India is the world's fifth-largest by nominal GDP."

### Let's Understand the working of Agents

#### Showcase the image and Explain the basic flow of Agent

### Now Built the Agent from the LangChain Version itself but this time we will use the Predefine Classes for the different type of Agents

In [48]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [49]:
search = TavilySearchResults()

In [50]:
search.invoke("what is the weather in SF")

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1727007490, 'localtime': '2024-09-22 05:18'}, 'current': {'last_updated_epoch': 1727007300, 'last_updated': '2024-09-22 05:15', 'temp_c': 15.6, 'temp_f': 60.1, 'is_day': 0, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/night/122.png', 'code': 1009}, 'wind_mph': 2.7, 'wind_kph': 4.3, 'wind_degree': 253, 'wind_dir': 'WSW', 'pressure_mb': 1014.0, 'pressure_in': 29.94, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 96, 'cloud': 100, 'feelslike_c': 15.6, 'feelslike_f': 60.1, 'windchill_c': 11.0, 'windchill_f': 51.9, 'heatindex_c': 11.9, 'heatindex_f': 53.3, 'dewpoint_c': 11.0, 'dewpoint_f': 51.7, 'vis_km': 13.0, 'vis_miles': 8.0, 'uv': 1.0, 'gust_mph': 4.7, 'gust_kph': 7.6}}"},
 {'url': 'https://www.timeanddate.com/weather/us

In [51]:
tools = [search]

#### We have different type of Agents

In [52]:
from langchain import hub

In [53]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

In [54]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [58]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

**agent_scratchpad:** The agent does not call a tool just once to get the desired answer, but has a structure that calls tools repeatedly until the desired answer is obtained. Each time you call a tool, what the previous call was like, information about the previous call, and the result are stored in this field.

In [55]:
from langchain.agents import create_tool_calling_agent
agent = create_tool_calling_agent(llm, tools, prompt)

#### Here explain about the AgentExecutor class

In [56]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [59]:
agent_executor.invoke({"input": "hello how are you?"})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `tavily_search_results_json` with `{'query': 'how are you'}`


[{'url': 'https://www.eslbuzz.com/how-are-you/', 'content': 'Here are some funny ways to say "how are you" in English: " What\'s shaking, bacon?". - This is a playful and informal way of asking "how are you.". It is often used jokingly or among friends and the word "bacon" is used as a friendly term of address. " How\'s your head, (fuzzy, clear, etc)?".'}, {'url': 'https://7esl.com/how-are-you/', 'content': '7ESL » Expressions » Phrases » 40 Other Ways to Ask “How Are You” in English\n40 Other Ways to Ask “How Are You” in English\nThe way that you ask someone “how are you?” may vary depending on who is being asked and in what setting the conversation is taking place. Related links:\n21 thoughts on “40 Other Ways to Ask “How Are You” in English”\nComment navigation\nWhat’s new\nI like lessons\nThank you\nGood\nTHANKS ALOT! How’s it rolling?\nComment navigation\nLeave a Comment Cancel reply\nComment\nΔ\nFree Course

{'input': 'hello how are you?', 'output': "I'm doing well, thanks for asking!"}

[{'url': 'https://www.eslbuzz.com/how-are-you/', 'content': 'Here are some funny ways to say "how are you" in English: " What\'s shaking, bacon?". - This is a playful and informal way of asking "how are you.". It is often used jokingly or among friends and the word "bacon" is used as a friendly term of address. " How\'s your head, (fuzzy, clear, etc)?".'}, {'url': 'https://7esl.com/how-are-you/', 'content': '7ESL » Expressions » Phrases » 40 Other Ways to Ask “How Are You” in English\n40 Other Ways to Ask “How Are You” in English\nThe way that you ask someone “how are you?” may vary depending on who is being asked and in what setting the conversation is taking place. Related links:\n21 thoughts on “40 Other Ways to Ask “How Are You” in English”\nComment navigation\nWhat’s new\nI like lessons\nThank you\nGood\nTHANKS ALOT! How’s it rolling?\nComment navigation\nLeave a Comment Cancel reply\nComment\nΔ\nFree Courses\nLevel Tests\nCompany\nFollow Us The list below shows different ways to ask “How Are You” in English that you can use in your daily English conversations.\n Idiomatic Ways to Ask “How Are You?”\nThese are playful, casual ways to find out how your buddy or pal is doing.'}, {'url': 'https://eslforums.com/how-are-you/', 'content': 'However, you can also use the question as a simple greeting and move on to a different topic of conversation.\nWays to Ask “How are you?”\nTo ask “how are you?”, you can use any of the phrases I provided in my previous responses. How Are You\nWays to Respond to “How are you?”\nTo answer the question, you can use any of the phrases I provided as well. Table of Contents\n“How are you?”\nHow to Use “How are you?”?\n or “I’ve been feeling a bit down, but I’m trying to focus on the good things in my life.”\nResponding to “How are you?” In this lesson, you will learn a list of 16 sentences you can use instead of “How are you?” in English and how to respond to it.'}, {'url': 'https://www.speakconfidentenglish.com/how-are-you-in-english/', 'content': '5 Ways to Ask ‘How Are You’ in Professional English Conversation\n3 Ways to Ask ‘How Are You’ When You Haven’t Seen Someone in a Long Time\nAfter you watch, I’d love to hear from you.\n When do they want the full story?\n12 Ways to Ask ‘How Are You’ in Casual English Conversation\n👉 To get in-depth details on what each question is really ask and how to best respond, be sure to watch the video lesson.\n ~Annemarie\nGet the Confidence to Say What You Want in English\nDownload my free training on how to build the courage and confidence you need to say what you want in English.\n Thank you for teaching us how to say “how are you “ in authentic English in this class and how to say “how are you “ for a small talk.\n #180: 20 Ways to Ask ‘How Are You’ in English [Expand Your Vocabulary]\nJul 1, 2020 | Advanced Vocabulary, Business Professional English, English Conversation\nLet’s talk about ‘How are you?”\n'}, {'url': 'https://www.grammarly.com/blog/how-are-you/', 'content': 'The key is understanding when to use the ubiquitous phrase, and when to try a different approach.\n9 ways to ask “How are you?”\nThere are many ways to ask this fundamental question, but how you phrase it can imply different things. This speaks directly to what you want to know, and it shows them that you understand that things may shift daily, while leaving it up to them about how detailed they want to get in their response.\n This approach can allow you to make an introduction via email that reminds them where you met (or last saw each other) and provides an opportunity for them to share their experience with a speaker or presenter you found interesting.\n Why it works: It’s polite, shows interest in their general well-being, and gets to the point (there’s a lot you don’t know, and you want the highlights).\n Because it’s phrased as a statement rather than a question, it allows the other person to skip the specifics if they don’t want to share personal details.\n'}]I'm doing well, thanks for asking!

In [60]:
agent_executor.invoke({"input": "whats the weather in sf?"})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `tavily_search_results_json` with `{'query': 'weather in san francisco'}`


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1727008670, 'localtime': '2024-09-22 05:37'}, 'current': {'last_updated_epoch': 1727008200, 'last_updated': '2024-09-22 05:30', 'temp_c': 15.6, 'temp_f': 60.1, 'is_day': 0, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/night/122.png', 'code': 1009}, 'wind_mph': 2.7, 'wind_kph': 4.3, 'wind_degree': 253, 'wind_dir': 'WSW', 'pressure_mb': 1014.0, 'pressure_in': 29.94, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 96, 'cloud': 100, 'feelslike_c': 15.6, 'feelslike_f': 60.1, 'windchill_c': 11.0, 'windchill_f': 51.9, 'heatindex_c': 11.9, 'heatindex_f': 53.3, 'dewpoint_c': 11.0, 'dewpoint_f': 51.7, 'vis_km': 13.0, 'vis_miles': 8.0, 'uv': 1.0, '

{'input': 'whats the weather in sf?',
 'output': 'It is currently 15.6 degrees Celsius (60.1 degrees Fahrenheit) and overcast in San Francisco. \n'}

### Lets add one more tool over here

In [61]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [62]:
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()

In [63]:
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en'}, page_content='\n\n\n\n\nGet started with LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\nSkip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith\u200bPythonTypeScriptpip install -U langsmithya

In [64]:
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)

In [65]:
vector = FAISS.from_documents(documents,embeddings)
retriever = vector.as_retriever()

In [66]:
retriever.invoke("how to upload a dataset")[0]

Document(metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en'}, page_content='description="A sample dataset in LangSmith.")client.create_examples(    inputs=[        {"postfix": "to LangSmith"},        {"postfix": "to Evaluations in LangSmith"},    ],    outputs=[        {"output": "Welcome to LangSmith"},        {"output": "Welcome to Evaluations in LangSmith"},    ],    dataset_id=dataset.id,)# Define your evaluatordef exact_match(run, example):    return {"score": run.outputs["output"] == example.outputs["output"]}experiment_results = evaluate(    lambda input: "Welcome " + input[\'postfix\'], # Your AI system goes here    data=dataset_name,

In [67]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [68]:
tools = [search, retriever_tool]

In [69]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

In [70]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [71]:
agent_executor.invoke({"input": "hi! what is a langsmith?"})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `langsmith_search` with `{'query': 'What is LangSmith?'}`


Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith​PythonTypeScriptpip install -U langsmithyarn add langsmith2. Create an API key​To create an API key head to the Settings page. Then click Create API Key.3. Set up your environment​Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-openai-api-key>4. Log your first trace​Tracing to LangSmith for LangChain usersThere is no need to use the Lang

{'input': 'hi! what is a langsmith?',
 'output': 'LangSmith is a platform for building production-grade LLM applications. \n'}

In [72]:
agent_executor.invoke({"input": "whats the weather in sf?"})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `tavily_search_results_json` with `{'query': 'weather in san francisco'}`


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1727008670, 'localtime': '2024-09-22 05:37'}, 'current': {'last_updated_epoch': 1727008200, 'last_updated': '2024-09-22 05:30', 'temp_c': 15.6, 'temp_f': 60.1, 'is_day': 0, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/night/122.png', 'code': 1009}, 'wind_mph': 2.7, 'wind_kph': 4.3, 'wind_degree': 253, 'wind_dir': 'WSW', 'pressure_mb': 1014.0, 'pressure_in': 29.94, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 96, 'cloud': 100, 'feelslike_c': 15.6, 'feelslike_f': 60.1, 'windchill_c': 11.0, 'windchill_f': 51.9, 'heatindex_c': 11.9, 'heatindex_f': 53.3, 'dewpoint_c': 11.0, 'dewpoint_f': 51.7, 'vis_km': 13.0, 'vis_miles': 8.0, 'uv': 1.0, '

{'input': 'whats the weather in sf?', 'output': '56 °F and overcast. \n'}

In [81]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [82]:
message_history = ChatMessageHistory()

In [83]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [87]:
agent_with_chat_history.invoke(
    {"input": "hi! my name is sunny how are you?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "firstchat"}},
)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


I am doing well, thank you for asking!  It's nice to meet you. 😊  How are you doing today?  


> Finished chain.


{'input': 'hi! my name is sunny how are you?',
 'chat_history': [HumanMessage(content='hi! my name is sunny', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Sunny is a gender-neutral name of English origin that means "sunshine," "happy," or "cheerful temperament." \n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content="what's my name?", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is Sunny.', additional_kwargs={}, response_metadata={})],
 'output': "I am doing well, thank you for asking!  It's nice to meet you. 😊  How are you doing today?  \n"}

In [89]:
agent_with_chat_history.invoke(
    {"input": "what's my name?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "firstchat"}},
)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Your name is Sunny. 


> Finished chain.


{'input': "what's my name?",
 'chat_history': [HumanMessage(content='hi! my name is sunny', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Sunny is a gender-neutral name of English origin that means "sunshine," "happy," or "cheerful temperament." \n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content="what's my name?", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is Sunny.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='hi! my name is sunny how are you?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I am doing well, thank you for asking!  It's nice to meet you. 😊  How are you doing today?  \n", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="what's my name?", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is Sunny. \n', additional_kwargs={}, response_metadata={})],
 'output': 'Your name is Sunny. \n'}

### Now Lets Start with ReAct Agent

##### **React agents and function calling agents are two distinct approaches to leveraging Large Language Models (LLMs) for automating tasks. Both methods aim to integrate LLMs with external tools and systems, but they differ in their underlying logic and implementation.**

**React Agent**

A React agent combines reasoning and action in a continuous loop. The LLM “thinks” about the problem, decides the steps to be taken, allows the agent to take action based on its reasoning, and then observes the result to refine its understanding. This approach relies on “ReACT prompting,” which involves crafting prompts that guide the LLM to alternate between reasoning and action steps.

##### Key features of React agents:

**1.Reasoning and action loop:** The agent continuously reasons about the task, decides on actions, and observes the outcomes to refine its understanding.

**2.ReACT prompting:** The agent uses specific prompting techniques to guide the LLM’s reasoning and action steps.

**3.Agent types:** React agents are tailored for specific tasks and utilize CoT (Conversational Task) and ReACT prompting.

**4.Strengths:** Suitable for tasks requiring multi-step reasoning, complex planning, and understanding of context.

**5.Limitations:** Can be computationally expensive due to the reasoning steps involved and requires more effort to define prompts and actions.

**Function Calling Agent**

A function calling agent, introduced by OpenAI, allows LLMs to understand a pre-defined format of a function (JSON structure) as part of a user prompt and intelligently choose to output a JSON object containing arguments to call a function (tool in an agent). This approach relies on pre-defined functions and their parameters.

##### Key features of function calling agents:

**1.Pre-defined functions:** The agent uses pre-defined functions with well-defined parameters and outputs.

**2.JSON-based input:** The agent receives JSON objects containing function names and arguments.

**3.Agent simplicity:** Function calling agents are generally easier to implement, relying on pre-defined functions and parameters.

**4.Strengths:** Excels at integrating LLMs with external systems and performing specific tasks through well-defined functions.

**5.Limitations:** Less suitable for open-ended tasks where actions are not pre-defined.
Comparison

In [90]:
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain_core.prompts import PromptTemplate
from langchain.agents import AgentExecutor, create_react_agent

In [91]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [92]:
SERPER_API_KEY=os.getenv("SERPER_API_KEY")
os.environ["SERPER_API_KEY"] = SERPER_API_KEY

In [93]:
google_search = GoogleSerperAPIWrapper()
tools = [
    Tool(
        name="Intermediate Answer",
        func=google_search.run,
        description="useful for when you need to ask with search",
        verbose=True
    )
]

In [94]:
template = '''Answer the following questions as best you can. You have access to the following tools:
{tools}
Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
Begin!
Question: {input}
Thought:{agent_scratchpad}'''

In [95]:
prompt = PromptTemplate.from_template(template)

In [96]:
search_agent = create_react_agent(llm,tools,prompt)

In [97]:
agent_executor = AgentExecutor(
    agent=search_agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,
)

In [98]:
response = agent_executor.invoke({"input": "Where is the hometown of the 2007 US PGA championship winner and his score?"})
print(response)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Thought: I need to know who won the 2007 US PGA Championship.
Action: Intermediate Answer
Action Input: Who won the 2007 US PGA Championship?Tiger Woods claimed his fourth PGA Championship and improved to 13–0 in major championships when holding at least a share of the 54-hole lead. He led by as many as five strokes in the third and fourth rounds.Thought: Now I need to find his hometown.
Action: Intermediate Answer
Action Input: Where is Tiger Woods from?Cypress, CAThought: I now know the final answer
Final Answer: Tiger Woods, the winner of the 2007 US PGA Championship, is from Cypress, CA. He won with a score of -12.  


> Finished chain.
{'input': 'Where is the hometown of the 2007 US PGA championship winner and his score?', 'output': 'Tiger Woods, the winner of the 2007 US PGA Championship, is from Cypress, CA. He won with a score of -12.', 'intermediate_steps': [(AgentAction(tool='Intermediate Answer', tool_input='Who won the 2007 US PGA Championship?', log='Thought: I need to kno

## ReAct Agent with Custom tools

In [107]:
from langchain.tools import tool
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent

In [108]:
# Custom tool for the Agent 
@tool
def get_employee_id(name):
  """
  To get employee id, it takes employee name as arguments
  name(str): Name of the employee
  """
  fake_employees = {
    "Alice": "E001",
    "Bob": "E002",
    "Charlie": "E003",
    "Diana": "E004",
    "Evan": "E005",
    "Fiona": "E006",
    "George": "E007",
    "Hannah": "E008",
    "Ian": "E009",
    "Jasmine": "E010"}
  
  return fake_employees.get(name,"Employee not found")

# Custom tool for the Agent 
@tool
def get_employee_salary(employee_id):
  """
  To get the salary of an employee, it takes employee_id as input and return salary
  """
  employee_salaries = {
    "E001": 56000,
    "E002": 47000,
    "E003": 52000,
    "E004": 61000,
    "E005": 45000,
    "E006": 58000,
    "E007": 49000,
    "E008": 53000,
    "E009": 50000,
    "E010": 55000
    }
  return employee_salaries.get(employee_id,"Employee not found")

In [109]:
# Saved React Prompt in langchain hub, we could manually type the prompt as well.
prompt = hub.pull("hwchase17/react")

In [110]:
tools = [get_employee_salary, get_employee_id]
agent = create_react_agent(llm,tools,prompt)
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor.invoke({"input":"What is the Salary of Evan?"})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Thought: I need to find Evan's employee ID first.
Action: get_employee_id
Action Input: EvanE005Thought: I need to find Evan's employee ID first.
Action: get_employee_id
Action Input: EvanE005Thought: Now that I have Evan's employee ID, I can find his salary.
Action: get_employee_salary
Action Input: E00545000Thought: I now know the final answer
Final Answer: 45000 


> Finished chain.


{'input': 'What is the Salary of Evan?', 'output': '45000'}

In [ ]:
agent_executor.invoke({"input":"What is the Salary of Evan?"})

# This given code from the latest version

https://python.langchain.com/docs/tutorials/agents/

In [101]:
# Import relevant functionality
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]}, config
):
    print(chunk)
    print("----")

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_b0f4', 'function': {'arguments': '{"query":"what is the weather like in san francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 92, 'prompt_tokens': 990, 'total_tokens': 1082, 'completion_time': 0.167272727, 'prompt_time': 0.031695343, 'queue_time': 0.0025771269999999985, 'total_time': 0.19896807}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1aa5a426-7415-432f-a325-2f46967ab1f4-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'what is the weather like in san francisco'}, 'id': 'call_b0f4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 990, 'output_tokens': 92, 'total_tokens': 1082})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {

## Create a custom agent

In [ ]:
from langchain.agents import tool


@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


get_word_length.invoke("abc")

In [ ]:
tools = [get_word_length]

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [ ]:
llm_with_tools = llm.bind_tools(tools)

In [ ]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [ ]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
list(agent_executor.stream({"input": "How many letters in the word eudca"}))

In [ ]:
llm.invoke("How many letters in the word educa")